In [1]:
# 참고자료 : https://velog.io/@choonsik_mom/pytorch%EB%A1%9C-LSTM-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0#-reference

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available() : 
    torch.cuda.manual_seed_all(125)
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (1.0,))
])

In [3]:
from torchvision.datasets import MNIST

download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, 
                      train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform,
                     train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform,
                    train=False, download=True)

100%|██████████| 9912422/9912422 [00:03<00:00, 3035166.54it/s]


Extracting ./MNIST_DATASET\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28834966.39it/s]


Extracting ./MNIST_DATASET\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3296477.63it/s]


Extracting ./MNIST_DATASET\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4527216.91it/s]

Extracting ./MNIST_DATASET\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_DATASET\MNIST\raw



In [4]:
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [5]:
batch_size=100
n_iters=6000
num_epochs=n_iters/(len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [54]:
import torch.nn.init as init

x2h = nn.Linear(1, 4, bias=False)
h2h = nn.Linear(1, 4, bias=False)

init.constant_(x2h.weight, 1.0) 
init.constant_(h2h.weight, 1.0) 

x2h(torch.Tensor([[1],[2]]))
# input_size = 1, hidden_size = 1


# hx, cx = hidden
x = torch.Tensor([[1]])
hx = torch.Tensor([[2]])
print(x.size())
x = x.view(-1, x.size(1))

gates = x2h(x) + h2h(hx)
print(gates)
print(gates.shape)
gates = gates.squeeze()
print(gates)

print(gates.chunk(4,1))
# ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)


torch.Size([1, 1])
tensor([[3., 3., 3., 3.]], grad_fn=<AddBackward0>)
torch.Size([1, 4])
tensor([3., 3., 3., 3.], grad_fn=<SqueezeBackward0>)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [49]:
import torch

# 2차원 tensor를 두 개의 2차원 tensor로 분할하는 예시
x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
x.squeeze_()
chunks = torch.chunk(x, 4)

# 분할된 chunk들 출력
for chunk in chunks:
    print(chunk)

tensor([[1, 2, 3]])
tensor([[4, 5, 6]])
tensor([[7, 8, 9]])
tensor([[10, 11, 12]])


In [73]:
class LSTMCell(nn.Module) :
    def __init__(self, input_size, hidden_size, bias=True) :
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4*hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias=bias)
        self.reset_parameters()
        
    def reset_parameters(self) :
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters() :
            w.data.uniform_(-std, std)
            
    def forward(self, x, hidden) :
        hx, cx = hidden
        print(x.shape)
        x = x.view(-1, x.size(1))
        print(x)
        print(x2h(x))
        print(h2h(hx))
        gates = self.x2h(x) + self.h2h(hx)
        print(gates)
        gates = gates.squeeze()
        print(gates)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4)
        
        ingate = F.sigmoid(ingate) # 입력 게이트에 시그모이드 적용
        forgetgate = F.sigmoid(forgetgate) # 망각 게이트에 시그모이드 적용
        cellgate = F.tanh(cellgate) # 셀 게이트에 탄젠트 적용
        outgate = F.sigmoid(outgate) # 출력 게이트에 시그모이드 적용
        
        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        
        return (hy, cy)

In [76]:
ex_model = LSTMCell(1,1,False)

input_data = torch.tensor([
    [[1.0], [2.0]], 
    [[3.0], [4.0]]
    ])  # (batch_size=2, sequence_length=2, input_size=1)
hidden_state = (torch.zeros(1, 2, 1), torch.zeros(1, 2, 1))  # 초기 은닉 상태와 셀 상태를 0으로 초기화
output = ex_model(input_data, hidden_state)  # 모델에 입력값 전달
print(output)

torch.Size([2, 2, 1])
tensor([[1., 2.],
        [3., 4.]])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2 and 1x4)

In [74]:
ex_model = LSTMCell(1,1,False)

input_data = torch.tensor([[1.0]])  # (batch_size=1, sequence_length=1, input_size=1)
hidden_state = (torch.zeros(1, 1), torch.zeros(1, 1))  # 초기 은닉 상태와 셀 상태를 0으로 초기화
output = ex_model(input_data, hidden_state)  # 모델에 입력값 전달
print(output)

output_2 = ex_model(input_data,output)
print(output_2)

torch.Size([1, 1])
tensor([[1.]])
tensor([[1., 1., 1., 1.]], grad_fn=<MmBackward0>)
tensor([[0., 0., 0., 0.]], grad_fn=<MmBackward0>)
tensor([[0.8278, 0.6969, 0.6420, 0.0942]], grad_fn=<AddBackward0>)
tensor([0.8278, 0.6969, 0.6420, 0.0942], grad_fn=<SqueezeBackward0>)
(tensor([[0.1962]], grad_fn=<MulBackward0>), tensor([[0.3940]], grad_fn=<AddBackward0>))
torch.Size([1, 1])
tensor([[1.]])
tensor([[1., 1., 1., 1.]], grad_fn=<MmBackward0>)
tensor([[0.1962, 0.1962, 0.1962, 0.1962]], grad_fn=<MmBackward0>)
tensor([[1.0131, 0.8765, 0.4767, 0.2892]], grad_fn=<AddBackward0>)
tensor([1.0131, 0.8765, 0.4767, 0.2892], grad_fn=<SqueezeBackward0>)
(tensor([[0.3086]], grad_fn=<MulBackward0>), tensor([[0.6036]], grad_fn=<AddBackward0>))


In [68]:
torch.zeros(1, 2, 1)

tensor([[[0.],
         [0.]]])

In [11]:
class LSTMModel(nn.Module) :
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True) :
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.layer_dim = layer_dim
        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x) :
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(self.layer_dim, x.size(0), hidden_dim))
           
        outs = []
        cn = c0[0,:,:]
        hn = h0[0,:,:]
            
        for seq in range(x.size(1)) :
            hn, cn = self.lstm(x[:, seq, :], (hn, cn))
            outs.append(hn)
            
        out = outs[-1].squeeze()
        out = self.fc(out)
        return out